<a href="https://colab.research.google.com/github/kamalahmadov474/MLOps/blob/main/Sklearn_Regression_Model_with_MLFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Part 1: Imports and Setup

In [ ]:
import warnings
import argparse
import logging
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
import os
import sys
import mlflow
import mlflow.sklearn

# Logging setup
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

# Command-line arguments
parser = argparse.ArgumentParser()
parser.add_argument("--alpha", type=float, required=False, default=0.5)
parser.add_argument("--l1_ratio", type=float, required=False, default=0.5)

# Prevent Jupyter from passing unwanted args
args, unknown = parser.parse_known_args(sys.argv[1:])

Part 2: Evaluation Function

In [ ]:
# Define function to evaluate model performance
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

Part 3: Main Program

In [ ]:
if __name__ == "__main__":
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Read the wine-quality csv file from local
    data = pd.read_csv(r"C:\Users\TUF\Downloads\red-wine-quality.csv")

    # Ensure "data" folder exists before saving
    os.makedirs("data", exist_ok=True)
    data.to_csv("data/red-wine-quality.csv", index=False)

Part 4: Data Preparation

In [ ]:
    # Split the data into training and test sets (75% train, 25% test)
    train, test = train_test_split(data)

    # Separate features (X) and target (y)
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]

Part 5: Model Training

In [ ]:
    # Get hyperparameters from arguments
    alpha = args.alpha
    l1_ratio = args.l1_ratio
    exp = mlflow.set_experiment(experiment_name="experiment_1")

    with mlflow.start_run(experiment_id = exp.experiment_id):

        # Initialize ElasticNet model
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)

        # Train model
        lr.fit(train_x, train_y)

2025/09/14 23:53:10 INFO mlflow.tracking.fluent: Experiment with name 'experiment_1' does not exist. Creating a new experiment.


Part 6: Prediction and Evaluation

In [ ]:
    # Make predictions
    predicted_qualities = lr.predict(test_x)

    # Evaluate predictions
    (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

    # Print results
    print("Elasticnet model (alpha={:f}, l1_ratio={:f}):".format(alpha, l1_ratio))
    print("  RMSE: %s" % rmse)
    print("  MAE: %s" % mae)
    print("  R2: %s" % r2)

    mlflow.log_param("alpha", alpha)
    mlflow.log_param("l1_ratio", l1_ratio)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mae", mae)
    mlflow.sklearn.log_model(lr, "mymodel")

2025/09/14 23:53:52 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.7931754052219389
  MAE: 0.6271747530439468
  R2: 0.1086008664957151


2025/09/14 23:53:55 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
